<a href="https://colab.research.google.com/github/chaufe/testcases/blob/main/colab/magic___buffer_overflow_when_merging_in_GF180MCU_SRAM_via_mag_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Testcase to show buffer-overflow in magic when creating GDS containing GF180MCU SRAM macro**

### Environment and Software Setup
- setup environment
- install conda-eda, GF180MCU PDK and EDA software
- install and patch helper software

In [ ]:
#@title Install a few packages required later-on { display-mode: "form" }
!dpkg --configure -a
!apt install python3-click

In [ ]:
#@title Install Anaconda environment {display-mode: "form"}
CONDA_PREFIX = '/content/eda'
%env CONDA_PREFIX = {CONDA_PREFIX}
!echo 'Will install CONDA EDA to {CONDA_PREFIX}'
!test -e Install.sh || wget -O Install.sh https://github.com/proppy/conda-eda/releases/download/v0.0-1445-gdbbed53/digital.gf180mcuc-0-Linux-x86_64.sh
!test -d {CONDA_PREFIX} || bash Install.sh -b -p {CONDA_PREFIX}

In [ ]:
#@title Update conda packages {display-mode: "form"}
!{CONDA_PREFIX}/bin/conda update --yes --all

In [ ]:
#@title Install EDA software { display-mode: "form" }


# use hard-coded packages as version resolution does not work as expected
for pkg in [
    'https://anaconda.org/LiteX-Hub/openroad/2.0_8931_g74ada9e7c/download/linux-64/openroad-2.0_8931_g74ada9e7c-20230606_125334.tar.bz2',
    'https://anaconda.org/LiteX-Hub/magic/8.3.409_0_gc8a2d06/download/linux-64/magic-8.3.409_0_gc8a2d06-20230606_125334.tar.bz2',
    'https://anaconda.org/LiteX-Hub/open_pdks.gf180mcuc/1.0.421_0_gb662727/download/noarch/open_pdks.gf180mcuc-1.0.421_0_gb662727-20230606_125334.tar.bz2'
]:
  print("Installing " + pkg)
  !{CONDA_PREFIX}/bin/conda install --yes {pkg}

In [ ]:
#@title Install klayout module to colab env { display-mode: "form" }
# Use colab's version of python as conda/openlane may use different python version
from platform import python_version_tuple
pversion = python_version_tuple()
!/usr/bin/python{pversion[0]+'.'+pversion[1]} -m pip install --upgrade --verbose --force-reinstall klayout

In [6]:
#@title Define function to directly display GDS { display-mode: "form"}
import pathlib
import klayout.db
import klayout.lay
import IPython.display
import os
import re

def display_gds_file(in_gds, zoom=None):
  lv = klayout.lay.LayoutView()
  lv.load_layer_props(os.environ.get('PDK_ROOT')+'/gf180mcuC/libs.tech/klayout/tech/gf180mcu.lyp')
  lv.load_layout(str(in_gds))
  lv.max_hier()
  via_layer_re = re.compile('^(Metal|Via)\d+$') # Only Metal* and Via* are visible
  itr = lv.begin_layers()
  while not itr.at_end():
    lp = itr.current()
    lp.visible = via_layer_re.match(lp.source_name)
    itr.next()
  #lv.set_config("background-color", "#FFFFFF")
  if zoom:
    lv.zoom_box(klayout.db.DBox(zoom[0], zoom[1], zoom[2], zoom[3]))
  lv.timer()
  pixels = lv.get_pixels_with_options(5000, 5000)
  IPython.display.clear_output(wait=True)
  return IPython.display.Image(pixels.to_png_data(), width=1500, height=1500)


In [ ]:
#@title Setup remaining variables { display-mode: 'form'}
%env CONDA_PREFIX = {CONDA_PREFIX}
import os
%env PATH = {CONDA_PREFIX + '/bin:' + os.getenv('PATH')}
%env PDK_ROOT = {CONDA_PREFIX + '/share/pdk'}
%env PDK = gf180mcuC
%env IO_LIB_PATH = {CONDA_PREFIX + '/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io'}
%env SRAM_LIB_PATH = {CONDA_PREFIX + '/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_ip_sram'}
!echo $PYTHONPATH

In [ ]:
#@title Create patch file for GF180MCU IO IP { display-mode: "form" }
#!wget https://raw.githubusercontent.com/chaufe/testcases/main/patches/gf180mcu_fd_io.lef.patch
%%writefile "gf180mcu_fd_io.lef.patch"
--- eda/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io/lef/gf180mcu_fd_io.lef	2023-02-13 13:43:43.000000000 +0100
+++ eda/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io/lef/gf180mcu_fd_io.lef	2023-03-30 10:19:34.792862000 +0200
@@ -3,6 +3,20 @@
   DIVIDERCHAR "/" ;
   BUSBITCHARS "[]" ;
+
+SITE GF_IO_Site
+  SYMMETRY X Y ;
+  CLASS PAD ;
+  SIZE 1 BY 350 ;
+END GF_IO_Site
+
+SITE GF_COR_Site
+  SYMMETRY X Y ;
+  CLASS PAD ;
+  SIZE 355 BY 355 ;
+END GF_COR_Site
+
+
 MACRO gf180mcu_fd_io__asig_5p0
-  CLASS PAD INOUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__asig_5p0 ;
   ORIGIN 0.000 0.000 ;
@@ -815,5 +829,5 @@

 MACRO gf180mcu_fd_io__bi_24t
-  CLASS PAD INOUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__bi_24t ;
   ORIGIN 0.000 0.000 ;
@@ -1669,5 +1683,5 @@

 MACRO gf180mcu_fd_io__bi_t
-  CLASS PAD INOUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__bi_t ;
   ORIGIN 0.000 0.000 ;
@@ -3807,5 +3821,5 @@

 MACRO gf180mcu_fd_io__dvdd
-  CLASS PAD POWER ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__dvdd ;
   ORIGIN 0.000 0.000 ;
@@ -4600,5 +4614,5 @@

 MACRO gf180mcu_fd_io__dvss
-  CLASS PAD POWER ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__dvss ;
   ORIGIN 0.000 0.000 ;
@@ -7609,5 +7623,5 @@

 MACRO gf180mcu_fd_io__in_c
-  CLASS PAD INPUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__in_c ;
   ORIGIN 0.000 0.000 ;
@@ -8416,5 +8430,5 @@

 MACRO gf180mcu_fd_io__in_s
-  CLASS PAD INPUT ;
+  CLASS PAD AREAIO ;
   FOREIGN gf180mcu_fd_io__in_s ;
   ORIGIN 0.000 0.000 ;


In [ ]:
#@title Patch GF180MCU IP { display-mode: "form" }
!patch -p0 < gf180mcu_fd_io.lef.patch

### Create Testcase

In [ ]:
#@title Write testcase RTL { display-mode: "form" }
%%writefile "testcase.v"
module testcase (input clk,
                 input din,
                 input rst,
                 output wire dout);

   wire clk_int;
   wire din_int;
   wire rst_int;
   wire dout_int;


   // pads
   (* keep *) gf180mcu_fd_io__bi_t pad_dout(
                                            .PU(1'b0), .PD(1'b0),
                                            .OE(1'b1), .IE(1'b0),
                                            .PDRV0(1'b0), .PDRV1(1'b0),
                                            .CS(1'b0), .SL(1'b0),
                                            .PAD(dout),
                                            .A(dout_int),
                                            .Y()
                                            );

   (* keep *) gf180mcu_fd_io__in_s pad_clk(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(clk),
                                           .Y(clk_int)
                                           );

   (* keep *) gf180mcu_fd_io__in_s pad_din(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(din),
                                           .Y(din_int)
                                           );

   (* keep *) gf180mcu_fd_io__in_s pad_rst(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(rst),
                                           .Y(rst_int)
                                           );

  // core-dump only occurs if macro placed in core area -> use SRAM
  wire [7:0] sram_q;
  (* keep *) gf180mcu_fd_ip_sram__sram512x8m8wm1 sram_i (
        .CLK (clk_int),
        .CEN (rst_int),
        .GWEN (rst_int),
        .WEN ({8{rst_int}}),
        .A ({9{din_d}}),
        .D ({8{din_d}}),
        .Q (sram_q),
  );

  reg din_d;
  reg din_d_xor;

    always @(posedge clk_int)
      if (rst_int) begin
        din_d <= 1'b0;
        din_d_xor <= 1'b0;
      end else begin
        din_d <= {din_d, din_int};
        din_d_xor <= ^{din_d,sram_q};
      end

  assign dout_int = din_d_xor;

endmodule


In [ ]:
#@title Write config.tcl to control OpenLANE { display-mode: "form" }
%%writefile "config.tcl"

set ::env(DESIGN_NAME) "testcase"

set ::env(VERILOG_FILES) [list "testcase.v" ]

set ::env(MACRO_PLACEMENT_CFG) "macro.cfg"

set ::env(EXTRA_LIBS) [list \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__tt_025C_5v00.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ff_n40C_5v50.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ff_125C_5v50.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ss_125C_4v50.lib \
  $::env(SRAM_LIB_PATH)/liberty/gf180mcu_fd_ip_sram__sram512x8m8wm1__tt_025C_5v00.lib \
  $::env(SRAM_LIB_PATH)/liberty/gf180mcu_fd_ip_sram__sram512x8m8wm1__ff_n40C_5v50.lib \
  $::env(SRAM_LIB_PATH)/liberty/gf180mcu_fd_ip_sram__sram512x8m8wm1__ff_125C_5v50.lib \
  $::env(SRAM_LIB_PATH)/liberty/gf180mcu_fd_ip_sram__sram512x8m8wm1__ss_125C_4v50.lib \
]

set ::env(EXTRA_LEFS) [list \
  $::env(SRAM_LIB_PATH)/lef/gf180mcu_fd_ip_sram__sram512x8m8wm1.lef \
]

#set ::env(BASE_SDC_FILE) "testcase.sdc"

set ::env(EXTRA_GDS_FILES) [list \
  $::env(IO_LIB_PATH)/gds/gf180mcu_fd_io.gds \
]

set ::env(DESIGN_IS_CORE) 1
set ::env(FP_PDN_CORE_RING) 1

set ::env(GRT_ALLOW_CONGESTION) 1
set ::env(DRT_OPT_ITERS) 10

set ::env(DIODE_INSERTION_STRATEGY) 0

set ::env(USE_GPIO_PADS) 1
set ::env(GPIO_PADS_LEF) $::env(IO_LIB_PATH)/lef/gf180mcu_fd_io.lef
unset ::env(GPIO_PADS_VERILOG)
set ::env(FP_IO_MODE) 1

set ::env(FP_PDN_UPPER_LAYER) Metal5
set ::env(FP_PDN_LOWER_LAYER) Metal4
set ::env(FP_PDN_RAILS_LAYER) Metal1
set ::env(FP_PDN_CORE_RING_VWIDTH) 5.0
set ::env(FP_PDN_CORE_RING_HWIDTH) 5.0
set ::env(FP_PDN_CORE_RING_VSPACING) 1.0
set ::env(FP_PDN_CORE_RING_HSPACING) 1.0

set ::env(SYNTH_NO_FLAT) 0
set ::env(SYNTH_SHARE_RESOURCES) 1

set ::env(CLOCK_TREE_SYNTH) 1
set ::env(CLOCK_PORT) "clk"
set ::env(CLOCK_PERIOD) 100

set ::env(FP_SIZING) absolute
set ::env(DIE_AREA) "0 0 1460 1460"
set ::env(CORE_AREA) "370 370 1090 1090"

set ::env(PL_TARGET_DENSITY) 0.65

# MAGIC GDS-out controls
## ... try GDS with hierarchy
set ::env(MAGIC_DISABLE_HIER_GDS) 0
set ::env(MAGIC_GENERATE_LEF) 0
set ::env(MAGIC_GENERATE_MAGLEF) 0

In [ ]:
#@title Create macro config for dummy SRAM { display-mode: "form" }
%%writefile 'macro.cfg'
sram_i  370 370 FS ;

In [ ]:
#@title Use updated ioplace script to create pad ring { display-mode: "form" }
%%writefile "eda/share/openlane/scripts/openroad/ioplacer.tcl"
# Copyright 2020-2022 Efabless Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
source $::env(SCRIPTS_DIR)/openroad/common/io.tcl
read

if { [info exists ::env(CONTEXTUAL_IO_FLAG)] } {
	read_lef $::env(placement_tmpfiles)/top_level.lef
}

if {$::env(FP_IO_HLENGTH) != "" && $::env(FP_IO_HLENGTH) != ""} {
	set_pin_length -hor_length $::env(FP_IO_HLENGTH) \
		-ver_length $::env(FP_IO_VLENGTH)
}

if {$::env(FP_IO_HEXTEND) != "0" && $::env(FP_IO_VEXTEND) != "0"} {
	set_pin_length_extension -hor_extension $::env(FP_IO_HEXTEND) \
		-ver_extension $::env(FP_IO_VEXTEND)
}

if {$::env(FP_IO_VTHICKNESS_MULT) != "" && $::env(FP_IO_HTHICKNESS_MULT) != ""} {
	set_pin_thick_multiplier -hor_multiplier $::env(FP_IO_HTHICKNESS_MULT) \
		-ver_multiplier $::env(FP_IO_VTHICKNESS_MULT)
}

set arg_list [list]
if { $::env(FP_IO_MODE) == 1 } {
	lappend arg_list -random
}

if { $::env(FP_IO_MIN_DISTANCE) != "" } {
	lappend arg_list -min_distance $::env(FP_IO_MIN_DISTANCE)
}

set HMETAL $::env(FP_IO_HLAYER)
set VMETAL $::env(FP_IO_VLAYER)

place_pins {*}$arg_list \
	-random_seed 42 \
	-hor_layers $HMETAL \
	-ver_layers $VMETAL

# create io-ring
make_io_sites -horizontal_site GF_IO_Site -vertical_site GF_IO_Site -corner_site GF_COR_Site -offset 0
place_pad -master gf180mcu_fd_io__in_s   -row IO_NORTH -location  400.0 pad_dout
place_pad -master gf180mcu_fd_io__in_s   -row IO_NORTH -location  700.0 pad_clk
place_pad -master gf180mcu_fd_io__dvss   -row IO_NORTH -location 1000.0 pad_dvss1

place_pad -master gf180mcu_fd_io__bi_24t -row IO_EAST  -location  400.0 pad_rst
place_pad -master gf180mcu_fd_io__in_s   -row IO_EAST  -location  700.0 pad_din
place_pad -master gf180mcu_fd_io__dvdd   -row IO_EAST  -location 1000.0 pad_dvdd1

place_pad -master gf180mcu_fd_io__dvdd   -row IO_WEST  -location  400.0 pad_dvdd2
place_pad -master gf180mcu_fd_io__dvss   -row IO_WEST  -location  700.0 pad_dvss2
place_pad -master gf180mcu_fd_io__dvdd   -row IO_WEST  -location 1000.0 pad_dvdd3

place_pad -master gf180mcu_fd_io__dvss   -row IO_SOUTH -location  400.0 pad_dvss3
place_pad -master gf180mcu_fd_io__dvdd   -row IO_SOUTH -location  700.0 pad_dvdd4
place_pad -master gf180mcu_fd_io__dvss   -row IO_SOUTH -location 1000.0 pad_dvss4

place_corners gf180mcu_fd_io__cor
foreach io_row { IO_EAST IO_WEST IO_SOUTH IO_NORTH } {
    place_io_fill -row $io_row gf180mcu_fd_io__fill10 gf180mcu_fd_io__fill5 gf180mcu_fd_io__fill1
}

connect_by_abutment

# work-around for pin-access error message

# get db/chip/block
set db [ord::get_db]
set chip [$db getChip]
set block [$chip getBlock]
set dbu_m [$block getDbUnitsPerMicron]

# mark all nets connected to pins (terminals) as special-net so OpenROAD will not try to route it
foreach bterm [$block getBTerms] {
  if {$bterm eq "NULL"} {
      error "Could not get terminals of block"
  } else {
    set net [$bterm getNet]
    if {$net eq "NULL"} {
      error "Block terminal [$bterm getName] returned NULL net"
    } else {
      puts "Setting net [$net getName] (connected to block terminal [$bterm getName]) to special-net"
      $net setSpecial
    }
  }
}

if {1} {
  make_net rst_int_buffered
  make_inst rst_int_buf gf180mcu_fd_sc_mcu7t5v0__buf_4
  foreach pin [get_pins -of_objects [get_nets rst_int] -filter "direction == input"] {
    puts "Reconnecting pin [get_full_name $pin] [get_property $pin direction]"
    disconnect_pin rst_int $pin
    connect_pin rst_int_buffered $pin
  }
  connect_pin rst_int_buffered rst_int_buf/Z
  connect_pin scan_enable_1 rst_int_buf/I
  set_dont_touch [get_net rst_int]
}

write


In [ ]:
#@title Run OpenLANE to create DEF for test case { display-mode: "form" }
%env PYTHONPATH = /usr/lib/python310.zip;/usr/lib/python3.10:/usr/lib/python3.10/lib-dynload:/usr/local/lib/python3.10/dist-packages:/usr/lib/python3/dist-packages:/usr/local/lib/python3.10/dist-packages/IPython/extensions
!flow.tcl -design . -to placement

In [ ]:
#@title Magic script to convert DEF to GDS (does trigger buffer overlow) { display-mode: "form" }
%%writefile def2gds.buffer_overflow.mag
lef read $::env(CONDA_PREFIX)/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_sc_mcu7t5v0/techlef/gf180mcu_fd_sc_mcu7t5v0__nom.tlef

add_path $::env(CONDA_PREFIX)/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_sc_mcu7t5v0/mag/

add_path $::env(SRAM_LIB_PATH)/mag/
#gds read $::env(SRAM_LIB_PATH)/gds/gf180mcu_fd_ip_sram__sram512x8m8wm1.gds

add_path $::env(CONDA_PREFIX)/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io/mag/

def read $::env(IN_DEF)
gds write $::env(IN_DEF).buffer_overflow.gds


In [ ]:
#@title Magic script to convert DEF to GDS (does NOT trigger buffer overlow) { display-mode: "form" }
%%writefile def2gds.no_buffer_overflow.mag
lef read $::env(CONDA_PREFIX)/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_sc_mcu7t5v0/techlef/gf180mcu_fd_sc_mcu7t5v0__nom.tlef

add_path $::env(CONDA_PREFIX)/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_sc_mcu7t5v0/mag/

#add_path $::env(SRAM_LIB_PATH)/mag/
gds read $::env(SRAM_LIB_PATH)/gds/gf180mcu_fd_ip_sram__sram512x8m8wm1.gds

add_path $::env(CONDA_PREFIX)/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io/mag/

def read $::env(IN_DEF)
gds write $::env(IN_DEF).no_buffer_overflow.gds


### Run Testcase

In [ ]:
#@title Merge GDS (trigger buffer overflow) { display-mode: "form" }
import pathlib
in_def = sorted(pathlib.Path('/content/runs').glob(f'*/results/placement/*.def'))[-1].resolve()
!IN_DEF={in_def} magic -dnull -noconsole -rcfile {CONDA_PREFIX}/share/pdk/gf180mcuC/libs.tech/magic/gf180mcuC.magicrc < /content/def2gds.buffer_overflow.mag | tee magic.log
!gzip -fv9 {in_def}.buffer_overflow.gds
!ln -sf {in_def}.buffer_overflow.gds.gz /content/


In [ ]:
#@title Merge GDS (NO buffer overflow) { display-mode: "form" }
import pathlib
in_def = sorted(pathlib.Path('/content/runs').glob(f'*/results/placement/*.def'))[-1].resolve()
!IN_DEF={in_def} magic -dnull -noconsole -rcfile {CONDA_PREFIX}/share/pdk/gf180mcuC/libs.tech/magic/gf180mcuC.magicrc < /content/def2gds.no_buffer_overflow.mag | tee magic.log
!gzip -fv9 {in_def}.no_buffer_overflow.gds
!ln -sf {in_def}.no_buffer_overflow.gds.gz /content/


In [ ]:
#@title Preview GDS using klayout {display-mode: "form"}
display_gds_file('/content/testcase.def.no_buffer_overflow.gds.gz')